In this We have taken from 2021-01-01 previous it wa 2021-09-01 for the start date of the in the train variable of the Model

And the forecasting is done for till July 2024

In [1]:
import pandas as pd
import numpy as np


# for Modelling
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

from tensorflow.keras.models import load_model

# for Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
df= pd.read_csv('aY_df_all.csv')
df

,Time id,Material,Material_Id,Quantity,Total_Value
0,7,S21-010448-04,41504.0,9.0,3615772.83
1,8,S21-010448-04,41504.0,19.0,7418193.30
2,9,S21-010448-04,41504.0,17.0,6579470.22
3,10,S21-010448-04,41504.0,9.0,3425975.42
4,11,S21-010448-04,41504.0,7.0,2673483.99
...,...,...,...,...,...
3158,35,AC37-000008,86.0,33.0,95237.05
3159,36,AC37-000008,86.0,44.0,156749.66
3160,37,AC37-000008,86.0,44.0,150426.96
3161,38,AC37-000008,86.0,35.0,119426.41


In [3]:
# Define a dictionary to map Time_id to dates
time_id_to_date = {
    1: "2021-01-01", 2: "2021-02-01", 3: "2021-03-01", 4: "2021-04-01", 5: "2021-05-01",
    6: "2021-06-01", 7: "2021-07-01", 8: "2021-08-01", 9: "2021-09-01", 10: "2021-10-01",
    11: "2021-11-01", 12: "2021-12-01", 13: "2022-01-01", 14: "2022-02-01", 15: "2022-03-01",
    16: "2022-04-01", 17: "2022-05-01", 18: "2022-06-01", 19: "2022-07-01", 20: "2022-08-01",
    21: "2022-09-01", 22: "2022-10-01", 23: "2022-11-01", 24: "2022-12-01", 25: "2023-01-01",
    26: "2023-02-01", 27: "2023-03-01", 28: "2023-04-01", 29: "2023-05-01", 30: "2023-06-01",
    31: "2023-07-01", 32: "2023-08-01", 33: "2023-09-01", 34: "2023-10-01", 35: "2023-11-01",
    36: "2023-12-01", 37: "2024-01-01", 38: "2024-02-01", 39: "2024-02-01"
}

# Change Time_id column to dates using the dictionary
df['Time id'] = df['Time id'].map(time_id_to_date)

# Optionally, convert the Time_id column to datetime type
df['Time id'] = pd.to_datetime(df['Time id'])

In [4]:
data = df.copy()

In [5]:
materials=data['Material'].unique()

In [6]:
# Define the evaluate_model function
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, r2

# Convert 'Time id' to datetime
data['Time id'] = pd.to_datetime(data['Time id'])

# Define train and test periods
train_start_date = '2021-01-01'
train_end_date = '2023-10-01'
test_start_date = '2023-11-01'
test_end_date = '2024-02-01'

# Define a function to build and train RNN models
def build_and_train_model(model_type, X_train, y_train):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material
for material in data['Material'].unique():
    print(f"Evaluating RNN models for Material: {material}")
    
    # Filter data for the current material
    material_data = data[data['Material'] == material]

    # Split data into train and test sets
    train = material_data[(material_data['Time id'] >= train_start_date) & (material_data['Time id'] <= train_end_date)]
    test = material_data[(material_data['Time id'] >= test_start_date) & (material_data['Time id'] <= test_end_date)]

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train[['Quantity']])
    test_scaled = scaler.transform(test[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    X_test = np.reshape(test_scaled, (test_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity
    y_test = test_scaled[:, 0]

    # Build and train Simple RNN model
    simple_rnn_model = build_and_train_model('SimpleRNN', X_train, y_train)

    # Build and train GRU model
    gru_model = build_and_train_model('GRU', X_train, y_train)

    # Build and train LSTM model
    lstm_model = build_and_train_model('LSTM', X_train, y_train)

    # Evaluate models on test set
    simple_rnn_mae, simple_rnn_r2 = evaluate_model(simple_rnn_model, X_test, y_test)
    gru_mae, gru_r2 = evaluate_model(gru_model, X_test, y_test)
    lstm_mae, lstm_r2 = evaluate_model(lstm_model, X_test, y_test)

    print("Simple RNN MAE:", simple_rnn_mae, " R2 Score:", simple_rnn_r2)
    print("GRU MAE:", gru_mae, " R2 Score:", gru_r2)
    print("LSTM MAE:", lstm_mae, " R2 Score:", lstm_r2)
    print("-" * 50)

Evaluating RNN models for Material: S21-010448-04


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
Simple RNN MAE: 0.15699098569651443  R2 Score: -8.94284359822521
GRU MAE: 0.2573686440785726  R2 Score: -25.768639829728386
LSTM MAE: 0.3081956108411153  R2 Score: -37.361664706354674
--------------------------------------------------
Evaluating RNN models for Material: S21-230126-06


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
Simple RNN MAE: 0.010260791000392719  R2 Score: 0.9712133221964009
GRU MAE: 0.051913273003366275  R2 Score: -0.050234896431876175
LSTM MAE: 0.06062828418281345  R2 Score: -0.15131480058575986
--------------------------------------------------
Evaluating RNN models for Material: S02-060098


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Simple RNN MAE: 0.01127256631851195  R2 Score: 0.9803437973735492
GRU MAE: 0.038988890051841724  R2 Score: 0.7113833788645723
LSTM MAE: 0.03885048806667327  R2 Score: 0.7182852123658403
--------------------------------------------------
Evaluating RNN models for Material: CB-YP01-20300


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Simple RNN MAE: 0.0395889086969968  R2 Score: 0.8027686184103278
GRU MAE: 0.07204660310827457  R2 Score: 0.44411627927023056
LSTM MAE: 0.046506198058868214  R2 Score: 0.675075665445592
--------------------------------------------------
Evaluating RNN models for Material: AC37-000008


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
Simple RNN MAE: 0.06386418734726156  R2 Score: 0.9448997945968691
GRU MAE: 0.1370990136736318  R2 Score: 0.7600468744939941
LSTM MAE: 0.15887404774364675  R2 Score: 0.6564814394209442
--------------------------------------------------


In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from datetime import timedelta

# Define the function to build and train the Simple RNN model
def build_and_train_simple_rnn(X_train, y_train):
    model = Sequential()
    model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Convert 'Time id' to datetime
data['Time id'] = pd.to_datetime(data['Time id'])

# Define the forecasting period
forecast_start_date = data['Time id'].max() + timedelta(days=1)
forecast_end_date = forecast_start_date + timedelta(days=6*30)  # Roughly 6 months
forecast_dates = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq='MS')

# Prepare the DataFrame to hold the forecast results
forecast_df = pd.DataFrame(columns=['Material', 'Time id', 'Forecasted Quantity'])

# Iterate over each material
for material in data['Material'].unique():
    print(f"Forecasting for Material: {material}")
    
    # Filter data for the current material
    material_data = data[data['Material'] == material]
    
    # Use all available data for training
    train = material_data[['Time id', 'Quantity']]
    
    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train[['Quantity']])
    
    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity
    
    # Build and train Simple RNN model
    simple_rnn_model = build_and_train_simple_rnn(X_train, y_train)
    
    # Forecast the next 6 months
    last_value = train_scaled[-1]
    forecasted_scaled_quantities = []
    
    for _ in range(len(forecast_dates)):
        next_input = np.reshape(last_value, (1, 1, 1))
        next_value_scaled = simple_rnn_model.predict(next_input)[0][0]
        forecasted_scaled_quantities.append(next_value_scaled)
        last_value = next_value_scaled
    
    # Inverse transform the forecasted values
    forecasted_quantities = scaler.inverse_transform(np.array(forecasted_scaled_quantities).reshape(-1, 1)).flatten()
    
    # Create a DataFrame for the current material's forecast
    material_forecast_df = pd.DataFrame({
        'Material': material,
        'Time id': forecast_dates,
        'Forecasted Quantity': forecasted_quantities
    })
    
    # Append the current material's forecast to the overall forecast DataFrame
    forecast_df = pd.concat([forecast_df, material_forecast_df], ignore_index=True)

print("Forecasting complete!")
print(forecast_df)


Forecasting for Material: S21-010448-04


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Forecasting for Material: S21-230126-06


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Forecasting for Material: S02-060098


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Forecasting for Material: CB-YP01-20300


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Forecasting for Material: AC37-000008


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Forecasting complete!
          Material    Time id  Forecasted Quantity
0    S21-010448-04 2024-03-01             8.259402
1    S21-010448-04 2024-04-01            11.894568
2    S21-010448-04 2024-05-01            14.395753
3    S21-010448-04 2024-06-01            16.108700
4    S21-010448-04 2024-07-01            17.277233
..             ...        ...                  ...
445    AC37-000008 2024-03-01            15.788293
446    AC37-000008 2024-04-01            23.388697
447    AC37-000008 2024-05-01            27.857710
448    AC37-000008 2024-06-01            30.462637
449    AC37-000008 2024-07-01            31.972034

[450 rows x 3 columns]
